In [1]:
import requests
import calendar
import time
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
def retriever(subreddit_1, subreddit_2, iterations):
    '''
    Retrieves the latest 500 (* iterations) posts and comments from r/subreddit_1 and r/subreddit_2
    '''
    
    utc_time = int(time.mktime(datetime.utcnow().timetuple()))

    params_1 = {'subreddit': f'{subreddit_1}',
                  #'filter': 'selftext, body, title, url, domain, created_utc, permalink',
                  'until': utc_time,
                  'limit': 500
                 }
              
    params_2 = {'subreddit': f'{subreddit_2}',
                 #'filter': 'selftext, body, title, url, domain, created_utc, permalink',
                 'until': utc_time,
                 'limit': 500
                }

    params = [params_1, params_2]

    urls = [{'subm': 'https://api.pushshift.io/reddit/search/submission/'},
            {'comm': 'https://api.pushshift.io/reddit/search/comment/'}]
    
    for i in range(len(params)):
        subreddit = params[i]
        sub_id = str(subreddit['subreddit'])[:3]
        url_type = ['subm', 'comm']
        
        for u in range(len(url_type)):
            
            for interval in range(iterations):
                res = requests.get(urls[u][url_type[u]], subreddit)
                
                if res.status_code == 200:
                    df = pd.DataFrame(res.json()['data'])
                    df.to_csv(f'../data/raw/df_{sub_id}_{url_type[u]}_0{interval}.csv', index = False)
                    subreddit['until'] = df['created_utc'][-1:]
                    print(f'Pull success :: {sub_id} ::{url_type[u]} :: _0{interval} :: {utc_time} :: {df.shape}')
                elif res.status_code != 200:
                    print(f'Pull failed :: status_code {res.status_code}')

In [10]:
retriever('geopolitics', 'worldnews', 2)

Pull success :: geo ::subm :: _00 :: 1674174186 :: (499, 92)
Pull success :: geo ::subm :: _01 :: 1674174186 :: (144, 93)
Pull success :: geo ::comm :: _00 :: 1674174186 :: (499, 55)
Pull success :: geo ::comm :: _01 :: 1674174186 :: (499, 55)
Pull success :: wor ::subm :: _00 :: 1674174186 :: (500, 92)
Pull success :: wor ::subm :: _01 :: 1674174186 :: (500, 92)
Pull success :: wor ::comm :: _00 :: 1674174186 :: (498, 53)
Pull success :: wor ::comm :: _01 :: 1674174186 :: (500, 53)
